### Imports

In [59]:
import os
import numpy as np
import pandas as pd

### Inputs

In [60]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_class = fusion_output_df['class'].values
fusion_output_confidence = fusion_output_df['confidence'].values

In [61]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Kalman Filter

In [62]:
def kalman_filter(z, Q=1e-4, R=1e-2):
    n = len(z)
    xhat = np.zeros(n)
    P = np.zeros(n)
    xhat[0] = z[0]
    P[0] = 1.0
    for k in range(1, n):
        # previsão
        xhatminus = xhat[k-1]
        Pminus = P[k-1] + Q
        # atualização
        K = Pminus / (Pminus + R)
        xhat[k] = xhatminus + K * (z[k] - xhatminus)
        P[k] = (1 - K) * Pminus
    return xhat

In [63]:
confidence_fusion = np.where(fusion_output_class == 1,
    fusion_output_confidence,
    1.0 - fusion_output_confidence
)

In [64]:
kalman = kalman_filter(confidence_fusion)

# classe final a partir do filtrado
cls = (kalman >= 0.5).astype(int)

# confiança orientada à classe escolhida
conf = np.where(cls == 1, kalman, 1.0 - kalman)

In [65]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}
for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-72    8.6   83     0.69   1        Bom      0.52      
-84    3.6   37     0.71   1        Bom      0.51      
-89    8.8   63     0.45   0        Ruim     0.55      
-36    3.0   22     0.20   1        Bom      0.60      
-58    4.8   53     0.75   1        Bom      0.58      
-57    1.5   22     0.37   1        Bom      0.64      
-83    7.4   89     0.37   1        Bom      0.61      
-67    6.5   86     0.51   1        Bom      0.59      
-50    3.6   23     0.26   1        Bom      0.64      
-68    5.2   40     0.90   1        Bom      0.60      
-87    9.6   65     0.66   1        Bom      0.56      
-32    3.0   23     0.21   1        Bom      0.60      
-62    4.2   87     0.92   1        Bom      0.58      
-67    3.2   36     0.32   1        Bom      0.59      
-65    3.0   36     0.22   1        Bom      0.62      
-37    2.5   31     0.29   1        Bom      0.6

### Save

In [66]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
